### With predicted rental prices, we are now interested in the growth rates for each SA2

In [29]:
import pandas as pd
import glob
import os

#path = r'../data/curated/2023_2027_rental_prediction/' # use your path
path = r'../data/curated/random_forest_pred/'
all_files = glob.glob(os.path.join(path , "*.csv"))

li = []

for filename in sorted(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

merged_df = pd.concat(li, axis=0, ignore_index=True)
predicted = merged_df.groupby(['year', 'sa2_2021'], as_index=False).agg({'predicted_price': 'mean'})
predicted

,year,sa2_2021,predicted_price
0,2023,201011001,425.086190
1,2023,201011002,448.302740
2,2023,201011005,370.642759
3,2023,201011006,448.057555
4,2023,201011007,474.777743
...,...,...,...
2490,2027,217031476,519.787713
2491,2027,217041477,419.761709
2492,2027,217041478,414.455290
2493,2027,217041479,419.982440


In [30]:
rental_2022 = pd.read_csv('../data/curated/merged_dataset/2022_merged_data.csv')
rental_2022.dropna(inplace=True)
rental_2022 = rental_2022.groupby(['year', 'sa2_2021'], as_index=False).agg({'weekly_rent': 'mean'})
rental_2022.rename({'weekly_rent': 'predicted_price'}, axis=1, inplace=True)
rental_2022
#rental_2022[rental_2022['sa2_2021']==206041127]
#set(rental_2022['sa2_2021'].unique()) - set(predicted['sa2_2021'].unique())

,year,sa2_2021,predicted_price
0,2022,201011001,437.468013
1,2022,201011002,421.270053
2,2022,201011005,402.559524
3,2022,201011006,440.000000
4,2022,201011007,530.625000
...,...,...,...
494,2022,217031476,518.750000
495,2022,217041477,414.069767
496,2022,217041478,771.301370
497,2022,217041479,437.025641


In [31]:
growth_df = pd.concat([rental_2022, predicted], axis=0).reset_index(drop=True)
#df[df['sa2_2021']==201011001]
growth_df

,year,sa2_2021,predicted_price
0,2022,201011001,437.468013
1,2022,201011002,421.270053
2,2022,201011005,402.559524
3,2022,201011006,440.000000
4,2022,201011007,530.625000
...,...,...,...
2989,2027,217031476,519.787713
2990,2027,217041477,419.761709
2991,2027,217041478,414.455290
2992,2027,217041479,419.982440


In [32]:
growth_dict = dict()
for sa2 in growth_df['sa2_2021'].unique():
    growth_dict[sa2] = []
    subset = growth_df[growth_df['sa2_2021']==sa2]
    for i in range(len(subset['predicted_price'])-1):
        # (current year price - past year price) / past year price
        growth_rate = (subset.iloc[i+1]['predicted_price'] - subset.iloc[i]['predicted_price'])/subset.iloc[i]['predicted_price']
        growth_dict[sa2].append(growth_rate)


In [33]:
growth_dict

{201011001: [-0.028303380118284035,
  0.017733765376069244,
  0.002876562388282571,
  -0.0023983912133289805,
  0.004988098514859138],
 201011002: [0.06416949374308864,
  0.003355573885698506,
  0.0011118041388257702,
  0.005151452581724591,
  0.052840100202371336],
 201011005: [-0.07928458409280122,
  0.006488886512348682,
  0.001492462073472579,
  0.002197368309460316,
  0.0026050709163990683],
 201011006: [0.0183126251345783,
  -0.04937048562737559,
  0.0010316461398479791,
  0.007574252403314079,
  -0.000818787684302797],
 201011007: [-0.10524807007261458,
  -0.04657870202148626,
  0.0031671987615281443,
  -0.014574328794960192,
  0.01988120091020559],
 201011008: [-0.022511523802300468,
  0.047504195793154215,
  0.025949562951140773,
  0.01299823238367695,
  0.00014460563916186535],
 201011481: [0.029922957932512657,
  -0.0006308370895259467,
  0.00960759446088438,
  0.014510935630644269,
  0.001187846881431064],
 201011482: [0.027420434407008726,
  0.013909486982580769,
  0.02453

In [34]:
import numpy as np
for key in growth_dict.keys():
    growth_dict[key] = np.mean(growth_dict[key])
growth_dict

{201011001: -0.0010206690104804124,
 201011002: 0.02532568491034177,
 201011005: -0.013300159256224117,
 201011006: -0.004654149926787606,
 201011007: -0.028670540243465454,
 201011008: 0.012817014592966667,
 201011481: 0.010919699563189285,
 201011482: 0.019707491206229964,
 201011483: 0.014395755293760623,
 201011484: 0.025229580953974172,
 201021009: 0.04302553175395167,
 201021010: 0.019999239776907603,
 201021011: -0.030227052502517605,
 201021012: 0.018011537056516327,
 201031013: 0.03425633848081955,
 201031014: 0.003398859392352962,
 201031015: 0.011154343767342373,
 201031016: 0.028266906876711885,
 201031017: 0.03370861090059032,
 202011018: 0.015808979967113748,
 202011019: -0.014940895542436893,
 202011020: 0.00636942863621425,
 202011021: -0.0008311030577055614,
 202011022: -0.009155526725543706,
 202011023: -0.003367870053950524,
 202011024: -0.012096249181821882,
 202011025: 0.0045680351708814616,
 202021026: 0.00910592980172477,
 202021027: 0.0020082206214931744,
 20202

In [35]:
# Sort dictionary so that the dictionary values are sorted as descneding order 
# The first subrub in the dictionary = the suburb which is predicted to have the highest average growth rate in the next 5 years 
sorted_dict = dict(sorted(growth_dict.items(), key=lambda item: item[1], reverse=True))
sorted_dict_list = list(sorted_dict)

sa2_list = list(sorted_dict.keys())
growth_list = list(sorted_dict.values())

top_ten_names = []

sa2_dict = pd.read_csv("../data/curated/sa2_vic_2021.csv")

i=0
while i < len(sa2_list):
    selected = sa2_list[i]
    suburb_name = sa2_dict[sa2_dict['SA2_CODE21']==selected]['SA2_NAME21'].to_list()[0]
    top_ten_names.append(suburb_name)
    i+=1

# Make a dataframe with the prediced results with columns of suburb name, its sa2 code and its growth rate
new_csv_name = "../data/curated/final_growing_rates_rf.csv" 

data = {'Suburb_Name': top_ten_names,
        'sa2_2021': sa2_list,
        'Growth_Rate': growth_list}

df = pd.DataFrame(data)
print(df)
df.to_csv(new_csv_name, index=False)

            Suburb_Name   sa2_2021  Growth_Rate
0    Narre Warren North  212021299     0.144086
1     Point Cook - East  213051464     0.139719
2                Elwood  206051129     0.102757
3          Yarriambiack  215011394     0.095828
4         Hawthorn East  207011152     0.084276
..                  ...        ...          ...
494     Bundoora - East  209011196    -0.071205
495      Sunshine North  213011337    -0.073282
496    Mansfield (Vic.)  204011057    -0.090158
497              Echuca  216011406    -0.091816
498        Moyne - West  217041478    -0.104430

[499 rows x 3 columns]
